In [76]:
import os
import cv2
import numpy as np
import os
import glob

# import SimpleITK as sitk
import scipy
import matplotlib.pyplot as plt
import pydicom
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pypinyin
import time

In [77]:
# 判断某文件是否是dicom格式的文件   :param filename: dicom文件的路 :return:'''
def isDicomFile(filename):
    with open(filename, "rb") as file_stream:
        file_stream.seek(128)
        data = file_stream.read(4)
    if data == b"DICM":
        return True
    return False


def readAllsubs(folder):  # 读取所有包含带slicelocation信息的文件夹。用于提取数据处理
    ddir = []
    for root, dirs, files in os.walk(folder):  # 检查每一个root下面是否有带attr的dicom文件
        dirlist = os.listdir(root)
        for ifile in dirlist:
            if os.path.isfile(os.path.join(root, ifile)):
                if isDicomFile(os.path.join(root, ifile)):
                    TheFile = pydicom.dcmread(os.path.join(root, ifile))
                    if hasattr(TheFile, "SliceLocation"):
                        ddir.append(root)
    return list(set(ddir))

In [93]:
def arrangeSlice(folderpath):
    files = []
    ppath = os.path.join(folderpath, "*")
    # print('glob: {}'.format(ppath ))
    for fname in glob.glob(ppath, recursive=False):
        # print("loading: {}".format(fname))
        if os.path.isfile(fname):
            if isDicomFile(fname):
                files.append(pydicom.dcmread(fname))
    # print("file count: {}".format(len(files)))
    # skip files with no SliceLocation (eg scout views)
    slices = []
    skipcount = 0
    for f in files:
        if hasattr(f, "SliceLocation"):
            # print(f['SliceLocation'])
            # print(f['SliceThickness'])
            slices.append(f)
        else:
            skipcount = skipcount + 1
    # print("skipped, no SliceLocation: {}".format(skipcount))
    try:
        slices = sorted(slices, key=lambda s: s.SliceLocation)
    except:
        slices = slices
        # print(folderpath)
    return slices


def get_pixels_hu(slices):
    N = len(slices)
    # assert the slice qualification
    list_slicelocation = [s.SliceLocation for s in slices]  ## 切片位置
    list_slicethickness = [s.SliceThickness for s in slices]  ## 切片厚度

    # if 1:
    #     assert(len(list_slicelocation)==len(set(list_slicelocation)))
    #     assert(len(set(list_slicethickness))==1)
    # if 1:
    # sshape=(512,512)
    # image = np.stack([cv2.resize(s.pixel_array,sshape) for s in slices])
    image = np.stack([s.pixel_array for s in slices])
    # if 0:
    # image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16),
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    # image[image == -2000] = 0
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept  ## 当前切片的截距
        slope = slices[slice_number].RescaleSlope  ## 当前切片的斜率
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
        image[slice_number] += np.int16(intercept)
    return np.array(image, dtype=np.int16)


def draw3DLinearTrans(image, HU_mean=-1200, HU_Diameter=1000):
    N = len(image)
    HU_upper = HU_mean + HU_Diameter * 0.5
    HU_lower = HU_mean - HU_Diameter * 0.5
    # assert the slice qualification
    for slice_number in range(N):
        image[slice_number][image[slice_number] > 3095] = 0
        image[slice_number] = image[slice_number] * (image[slice_number] > HU_lower) * (image[slice_number] < HU_upper) + HU_upper * (image[slice_number] > HU_upper) + HU_lower * (image[slice_number] < HU_lower)
    return image, HU_lower, HU_upper


def resample(image, scan, new_spacing=[1, 1, 1]):
    # scan是原始的slices
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]], dtype=np.float32)
    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode="nearest")
    return image, new_spacing


def segment_lung_mask(image, fill_lung_structures=True):
    # not actually binary, but 1 and 2.
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8) + 1  ## 图像变为 1 and 2
    labels = measure.label(binary_image)  ## 连通域
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air
    #   around the person in half
    background_label = labels[0, 0, 0]  ## 最上角一定是空气
    # Fill the air around the person
    binary_image[background_label == labels] = 2  ## 跟空气连接在一起的判定为空气
    # Method of filling the lung structures (that is superior to something like
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1  ## 0 1
            labeling = measure.label(axial_slice)  ## 当前slice的连通域
            l_max = largest_label_volume(labeling, bg=0)  ### 数量最多的标签
            if l_max is not None:  # This slice contains some lung
                binary_image[i][labeling != l_max] = 1
    binary_image -= 1  # Make the image actual binary 0 1
    binary_image = 1 - binary_image  # Invert it, lungs are now 1
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None:  # There are air pockets
        binary_image[labels != l_max] = 0
    return binary_image


def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)
    counts = counts[vals != bg]
    vals = vals[vals != bg]  ## 不是-1的值
    if len(counts) > 0:
        return vals[np.argmax(counts)]  ## 数量最多的值
    else:
        return None

def save_3d(image,foldername,threshold=400):
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    verts, faces, normals, values = measure.marching_cubes_lewiner(p, threshold)  #### ？？？？
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.70)
    face_color = [0.45, 0.45, 0.75]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)
    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])  
    ax.set_zlim(0, p.shape[2])
    pngname = foldername.replace(":","").replace("\\","-").replace("/","-")+".png"
    plt.savefig(pngname)
    plt.close("all")
    print("plot 3d image down")

In [98]:
sourceFolder = "RuiTestData/正常结核/2881120/46A14F0033A6479BA7A7D85AF21DCF4"
myslices = arrangeSlice(sourceFolder)
myimage2 = get_pixels_hu(myslices)
myimage, u1, u2 = draw3DLinearTrans(myimage2, HU_mean=-700, HU_Diameter=1000)

In [97]:
path1='Desktop/luna16/test'
if myimage.shape[0] > 10:
    if int(myslices[0].SliceThickness) <= 10:
        myimage, new_spacing = resample(myimage, myslices, new_spacing=[1, 1, 1])
        print(myimage.shape)
        segmented_lungs_fill = segment_lung_mask(myimage, True)
        save_3d(segmented_lungs_fill, path1, threshold=0)
        plt.close()

(344, 324, 324)


/Users/chromium/opt/anaconda3/envs/MedImg/lib/python3.7/site-packages/ipykernel_launcher.py:131: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19


plot 3d image down


In [ ]:
myslices